In [2]:
# usa el archivo en formato wide que contiene fraudes y no fraudes (generado por la notebook: create_fraud_normal_format)

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import roc_auc_score, roc_curve
import seaborn as sns
from sklearn.model_selection import train_test_split
import joblib

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
# from src.preprocessing.preprocessing  import data_wrangling
from src.modeling.supervised_models import LGBMModel, CATModel, compute_stacking_model
from src.modeling.helpers import save_model

2023-11-12 20:11:03.181825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-12 20:11:03.289284: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-11-12 20:11:03.289385: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-11-12 20:11:03.320881: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.


## Configuracion

In [12]:
DATA_PATH_INPUT = '../../data/final/'
MODEL_PATH = '../../models/'

file_name_train_data = 'train_data_final_tsfel.parquet'
file_cols_for_model = 'features.pkl'

file_lgbm_model_name = 'lgbm_model.pkl'
file_catboost_model_name = 'catboost_model.pkl'
file_stacking_model_name = 'combination_final_model.pkl'

sam_th,param_imb_method,preprocesor_num =1.0, 'over',3

## Train models

In [13]:
df = pd.read_parquet(DATA_PATH_INPUT+file_name_train_data)
cols_for_model = joblib.load(MODEL_PATH+file_cols_for_model)

In [14]:
df_train_f, df_val_f, y_trainf, y_val_f = train_test_split(df.drop(columns=['target']),df.target, test_size=0.1, random_state=42)

In [15]:
final_lgbm_model = LGBMModel(cols_for_model,hyperparams=None,search_hip=False,sampling_th = sam_th,preprocesor_num = preprocesor_num,sampling_method=param_imb_method)
final_lgbm_model = final_lgbm_model.train(df_train_f,y_trainf)
save_model(final_lgbm_model,MODEL_PATH+file_lgbm_model_name)

In [16]:
final_cb_model = CATModel(cols_for_model,hyperparams=None,search_hip=False,sampling_th = sam_th,preprocesor_num = preprocesor_num,sampling_method=param_imb_method)
final_cb_model = final_cb_model.train(df_train_f,y_trainf)
save_model(final_cb_model,MODEL_PATH+file_catboost_model_name)

In [17]:
combination_final = compute_stacking_model(final_lgbm_model,final_cb_model,df_val_f,y_val_f,cols_for_model)
save_model(combination_final,MODEL_PATH+file_stacking_model_name)